In [1]:
import numpy as np 
import pandas as pd 
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from skimage import io

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50 
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

In [2]:
train_df = pd.read_csv('train.csv')
valid_df = pd.read_csv('test.csv')

## Setting up the image augmentation from last Lesson: 
Note that this section of the code has been pre-written for you and does not need to be changed, just run. If you would like to change the ImageDataGenerator parameters, feel free.

In [3]:
## This is the image size that VGG16 takes as input
IMG_SIZE = (224, 224)

In [4]:
train_idg = ImageDataGenerator(rescale=1. / 255.0,
                              horizontal_flip = True, 
                              vertical_flip = False, 
                              height_shift_range= 0.1, 
                              width_shift_range=0.1, 
                              rotation_range=20, 
                              shear_range = 0.1,
                              zoom_range=0.1)

train_gen = train_idg.flow_from_dataframe(dataframe=train_df, 
                                         directory=None, 
                                         x_col = 'img_path',
                                         y_col = 'class',
                                         class_mode = 'binary',
                                         target_size = IMG_SIZE, 
                                         batch_size = 9
                                         )

# Note that the validation data should not be augmented! We only want to do some basic intensity rescaling here
val_idg = ImageDataGenerator(rescale=1. / 255.0)

val_gen = val_idg.flow_from_dataframe(dataframe=valid_df, 
                                         directory=None, 
                                         x_col = 'img_path',
                                         y_col = 'class',
                                         class_mode = 'binary',
                                         target_size = IMG_SIZE, 
                                         batch_size = 6) ## We've only been provided with 6 validation images

Found 20 validated image filenames belonging to 2 classes.
Found 6 validated image filenames belonging to 2 classes.


In [5]:
## Pull a single large batch of random validation data for testing after each epoch
testX, testY = val_gen.next()

In [7]:
testX.shape

(6, 224, 224, 3)

## Now we'll load in VGG16 with pre-trained ImageNet weights: 

In [8]:
model = VGG16(include_top=True, weights='imagenet')
model.summary()

553467904/553467096 [==============================] - 49s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
___________________________________________________________

In [10]:
transfer_layer = model.get_layer('block5_pool')
vgg_model = Model(inputs=model.input,
                   outputs=transfer_layer.output)

In [11]:
## Now, choose which layers of VGG16 we actually want to fine-tune
## Here, we'll freeze all but the last convolutional layer

## Add some code here to freeze all but the last convolutional layer:
##### Your code here ######
for layer in vgg_model.layers[0:17]:
    layer.trainable = False

In [12]:
## Check to make sure you froze the right ones: 
for layer in vgg_model.layers:
    print(layer.name, layer.trainable)

input_1 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 True
block5_pool True


## Build a simple sequential model using only the VGG16 architecture
Note the code in the cell below has been pre-written for you, you only need to run it

In [13]:
## Build your model using the mostly-frozen VGG16 architecture: 
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(1, activation='relu'))

In [14]:
## Set our optimizer, loss function, and learning rate (you can change the learning rate here if you'd like)
## but otherwise this cell can be run as is
optimizer = Adam(lr=1e-4)
loss = 'binary_crossentropy'
metrics = ['binary_accuracy']

In [15]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [16]:
## Just run a single epoch to see how it does:
new_model.fit_generator(train_gen, 
                                  validation_data = (testX, testY), 
                                  epochs = 5)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
3/3 [==============================] - 2s 769ms/step - loss: 3.9158 - binary_accuracy: 0.3000 - val_loss: 0.9835 - val_binary_accuracy: 0.5000
Epoch 2/5
3/3 [==============================] - 2s 686ms/step - loss: 5.9372 - binary_accuracy: 0.5000 - val_loss: 0.6372 - val_binary_accuracy: 0.6667
Epoch 3/5
3/3 [==============================] - 3s 1s/step - loss: 1.6139 - binary_accuracy: 0.6000 - val_loss: 7.7125 - val_binary_accuracy: 0.5000
Epoch 4/5
3/3 [==============================] - 2s 750ms/step - loss: 2.7563 - binary_accuracy: 0.5500 - val_loss: 7.7125 - val_binary_accuracy: 0.5000
Epoch 5/5
3/3 [==============================] - 3s 1s/step - loss: 7.0230 - binary_accuracy: 0.5000 - val_loss: 7.7125 - val_binary_accuracy: 0.5000


## Let's try another experiment where we add a few more dense layers:

In [17]:
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(1024, activation='relu'))

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(512, activation='relu'))

# Add a dense (aka. fully-connected) layer.
# Change the activation function to sigmoid 
# so output of the last layer is in the range of [0,1] 
new_model.add(Dense(1, activation='sigmoid'))

In [18]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [19]:
## Just run a single epoch to see how it does:
new_model.fit_generator(train_gen, 
                                  validation_data = (testX, testY), 
                                  epochs = 5)

Epoch 1/5
3/3 [==============================] - 2s 822ms/step - loss: 1.3666 - binary_accuracy: 0.1500 - val_loss: 0.7535 - val_binary_accuracy: 0.5000
Epoch 2/5
3/3 [==============================] - 3s 1s/step - loss: 0.8098 - binary_accuracy: 0.5000 - val_loss: 0.9757 - val_binary_accuracy: 0.5000
Epoch 3/5
3/3 [==============================] - 2s 762ms/step - loss: 0.8774 - binary_accuracy: 0.5000 - val_loss: 0.7274 - val_binary_accuracy: 0.5000
Epoch 4/5
3/3 [==============================] - 3s 1s/step - loss: 0.4964 - binary_accuracy: 0.9500 - val_loss: 0.6480 - val_binary_accuracy: 0.6667
Epoch 5/5
3/3 [==============================] - 2s 774ms/step - loss: 0.5519 - binary_accuracy: 0.7000 - val_loss: 0.6803 - val_binary_accuracy: 0.6667


## Now let's add dropout and another fully connected layer:

In [20]:
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
new_model.add(Dropout(0.5))

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(1024, activation='relu'))

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
new_model.add(Dropout(0.5))

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(512, activation='relu'))

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
new_model.add(Dropout(0.5))

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(256, activation='relu'))

# Add a dense (aka. fully-connected) layer.
# Change the activation function to sigmoid 
# so output of the last layer is in the range of [0,1] 
new_model.add(Dense(1, activation='sigmoid'))

In [21]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [22]:
## Just run a single epoch to see how it does:
new_model.fit_generator(train_gen, 
                                  validation_data = (testX, testY), 
                                  epochs = 5)

Epoch 1/5
3/3 [==============================] - 3s 1s/step - loss: 0.8813 - binary_accuracy: 0.4000 - val_loss: 0.6765 - val_binary_accuracy: 0.5000
Epoch 2/5
3/3 [==============================] - 3s 1s/step - loss: 0.7052 - binary_accuracy: 0.6000 - val_loss: 0.6448 - val_binary_accuracy: 0.8333
Epoch 3/5
3/3 [==============================] - 3s 1s/step - loss: 0.9197 - binary_accuracy: 0.3500 - val_loss: 0.6396 - val_binary_accuracy: 0.6667
Epoch 4/5
3/3 [==============================] - 2s 759ms/step - loss: 0.7892 - binary_accuracy: 0.6000 - val_loss: 0.6224 - val_binary_accuracy: 0.6667
Epoch 5/5
3/3 [==============================] - 2s 779ms/step - loss: 0.7412 - binary_accuracy: 0.6000 - val_loss: 0.6906 - val_binary_accuracy: 0.5000
